In [ ]:
!pip install scikit-learn
!python --version

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import keras

from keras import layers
from keras.utils import timeseries_dataset_from_array, to_categorical
import logging
from enum import Enum
import itertools
import os
import pickle
import re
import random
import time

from sklearn.preprocessing import MinMaxScaler
#from utils import load_gesture_samples, GestureNames, split_data_between_participants, normalize_dataset

In [ ]:
# Setup logger
logging.basicConfig(level=logging.DEBUG,
                    force = True)

log = logging.getLogger("CSE3000")
log.setLevel(logging.DEBUG)

np.random.seed(1000)
tf.random.set_seed(1000)


In [ ]:
# Utility functions cell
class Hand(Enum):
    right = "right_hand"
    left = "left_hand"


class GestureException(Exception):
    pass


class Gestures(Enum):
    SWIPE_LEFT = 0, 'swipe_left'
    SWIPE_RIGHT = 1, 'swipe_right'
    SWIPE_UP = 2, 'swipe_up'
    SWIPE_DOWN = 3, 'swipe_down'
    ROT_CW = 4, 'clockwise'
    ROT_CCW = 5, 'counter_clockwise'
    TAP = 6, 'tap'
    DOUBLE_TAP = 7, 'double_tap'
    ZOOM_IN = 8, 'zoom_in'
    ZOOM_OUT = 9, 'zoom_out'

    def __new__(cls, value, name):
        member = object.__new__(cls)
        member._value_ = value
        member.fullname = name
        return member

    def __int__(self):
        return self.value

    @staticmethod
    def from_name(name: str):
        try:
            return next(g for g in Gestures if g.fullname == name)
        except StopIteration:
            raise GestureException("No gesture with name '%s' found..." % name)


class LoadGestureException(Exception):
    pass


def load_gesture_samples(gesture: Gestures, hand: Hand = Hand.right, skip_old_data: bool = True):
    result = []
    base_path = f"gestures_data/gestures/{gesture.fullname}/{hand.value}"
    log.debug("Loading gestures from base path: %s" % base_path)
    folder_items = os.listdir(base_path)

    # Filter on the .pickle extension
    filtered_ext = list(filter(lambda x: re.search(r'\.pickle$', x) is not None, folder_items))

    if len(filtered_ext) == 0:
        raise LoadGestureException("No gestures found in folder: %s" % base_path)

    for item in filtered_ext:
        r_match = re.match(r'candidate_(\w+).pickle$', item)
        if r_match is None:
            raise LoadGestureException("Incorrectly formatted data file name: %s" % item)

        candidate_id = r_match.group(1)
        with open(os.path.join(base_path, item), 'rb') as f:
            while True:
                try:
                    data_contents = pickle.load(f)

                    if isinstance(data_contents, dict):
                        if 'target_gesture' in data_contents:
                            # Data v3
                            # print(data_contents)
                            data_contents['gesture'] = Gestures.from_name(data_contents['target_gesture'])
                            # data_contents['all_data'] = data_contents['data']
                            # print(type(data_contents['data']))
                            # data_contents['data'] = list(map(lambda x: x['data'], data_contents['data']))
                            result.append(data_contents)
                        else:
                            # Data v2
                            data_contents['gesture'] = Gestures.from_name(data_contents['gesture'])
                            if not skip_old_data:
                                result.append(data_contents)
                    else:
                        # Data loader v1
                        data = {
                            'data': data_contents,
                            'gesture': gesture,
                            'candidate': candidate_id
                        }
                        if not skip_old_data:
                            result.append(data)
                except EOFError:
                    break

    return result


def split_data_between_participants(data, ratio = 0.7):
    lb_candidate = lambda x: x['candidate']

    # For itertools.groupby to work we need to sort the data first
    data.sort(key=lambda x: x['candidate'])

    participants = set(map(lb_candidate, data))
    amount_measurements = len(data)
    amount_participants = len(participants)

    log.debug("Participants: %s" % participants)
    log.debug("Got dataset for %d participants with %d measurements total" % (amount_participants, amount_measurements))

    amount_train = int(amount_measurements * 0.7)
    amount_test = amount_measurements - amount_train
    log.debug("Estimating %d measurements for training and %d measurements for test (ratio: %0.1f)" % (amount_train, amount_test, ratio))

    train_data = []
    train_data_outcomes = []

    test_data = []
    test_data_outcomes = []

    train_candidates = set()
    test_candidates = set()

    # Group the data per participant as that is the recommended method for training models
    for participant, d in itertools.groupby(data, lb_candidate):
        d_list = list(d)

        for data_point in d_list:
            if len(train_data) < amount_train:
                train_candidates.add(participant)
                train_data.append(data_point['data'])
                train_data_outcomes.append(data_point['gesture'])
            else:
                test_candidates.add(participant)
                test_data.append(data_point['data'])
                test_data_outcomes.append(data_point['gesture'])
                # test_data.extend([p['data'] for p in d_list])
                # test_data_outcomes.extend([p['gesture'].value for p in d_list])

    log.debug("Train candidates: %s\tTest candidates: %s" % (train_candidates, test_candidates))

    return (np.array(train_data), np.array(train_data_outcomes)), (np.array(test_data), np.array(test_data_outcomes))

def normalize_dataset(data):
    """Watch out this function might not normalize the data as expected, further research required"""
    normalized = []
    for graph in data:
        scaler = MinMaxScaler(feature_range=(0, 1))
        reshaped = scaler.fit_transform(graph.reshape(-1, graph.shape[-1])).reshape(graph.shape)
        normalized.append(reshaped)
    return np.array(normalized)

def normalize_test(data):
    normalized = []
    for measurement in data:
        mean = measurement.mean()
        std = measurement.std()
        normalized_measurement = (measurement - mean) / std

        normalized.append(normalized_measurement)
    return normalized

def plot_history(history):
    loss = history.history['loss']
    mae = history.history['mae']

    epochs = range(1, len(loss) + 1)

    fig, axs = plt.subplots(1, 2, figsize=(15, 5))

    axs[0].plot(epochs, loss, 'g.', label='Training Loss')
    axs[0].set_xlabel('Epochs')
    axs[0].set_ylabel('Loss')
    axs[0].legend()

    axs[1].plot(epochs, mae, 'g.', label='Training Accuracy')
    axs[1].set_xlabel('Epochs')
    axs[1].set_ylabel('MAE')
    axs[1].legend()

    fig.show()

In [ ]:
combined = []

for g in Gestures:
    samples = load_gesture_samples(g)
    combined.extend(samples)

# Deterministic shuffle
random.Random(4).shuffle(combined)
(x_train, y_train), (x_test, y_test) = split_data_between_participants(combined)

print(x_train.shape)
print(y_train.shape)

print(x_test.shape)
print(y_test.shape)

x_train_normalized = normalize_dataset(x_train)
x_test_normalized = normalize_dataset(x_test)
y_train = to_categorical(y_train, len(Gestures))
y_test = to_categorical(y_test, len(Gestures))

# Hardcoded for now, so fix
x_train_normalized = x_train_normalized.reshape((-1, 100, 3, 1))
x_test_normalized = x_test_normalized.reshape((-1, 100, 3, 1))

print(x_train_normalized.shape)
print(x_test_normalized.shape)


In [ ]:
# Show all training data
train_amount = len(x_train)
fig, axs = plt.subplots(1, train_amount, figsize=(40, 4))
for i in range(train_amount):
    axs[i].plot(x_train[i])
    axs[i].annotate(Gestures(y_train[i]).fullname, (0,0), (0, -40), xycoords='axes fraction', textcoords='offset points', va='top')

In [ ]:
# Show all test data
test_amount = len(x_test)
fig, axs = plt.subplots(1, test_amount, figsize=(20, 4))
for i in range(test_amount):
    axs[i].plot(x_test[i])
    axs[i].annotate(f'Gesture: {Gestures(y_test[i]).fullname}', (0,0), (0, -40), xycoords='axes fraction', textcoords='offset points', va='top')

In [ ]:
# Show normalized test data
test_amount = len(x_test_normalized)
fig, axs = plt.subplots(1, test_amount, figsize=(20, 4))
for i in range(test_amount):
    axs[i].plot(x_test_normalized[i])

In [ ]:
# Implement data windowing
# SLICES = 5
#
# single_source = np.array(train[0], dtype=np.uint16)
# single_targets = np.full((single_source.shape[0], 1, 1), train_outcome[0])
# print(single_source.shape)
#
# train_ds = timeseries_dataset_from_array(data=single_source, targets=single_targets, sequence_length=SLICES, sequence_stride=SLICES)
#
# for example_inputs, example_labels in train_ds.take(1):
#     print(f'Inputs shape (batch, time, features): {example_inputs.shape}')
#     print(f'Targets shape (batch, time, features): {example_labels.shape}')
#
#
# single_validation_source = np.array(test[0], dtype=np.uint16)
# single_validation_targets = np.full((single_validation_source.shape[0], 1, 1), test_outcome[0])
#
# validation_ds = keras.utils.timeseries_dataset_from_array(data=single_validation_source, targets=single_validation_targets, sequence_length=SLICES, sequence_stride=SLICES)


In [ ]:
# train_ds = timeseries_dataset_from_array(data=x_train, targets=y_train, sequence_length=SLICES, sequence_stride=SLICES)
# validation_ds = timeseries_dataset_from_array(data=x_test, targets=y_test, sequence_length=SLICES, sequence_stride=SLICES)

In [ ]:
# print(dir(train_ds))
# print(train_ds.element_spec)

In [ ]:
USE_GPU = False
device = "/device:GPU:0" if USE_GPU else "/device:CPU:0"

with tf.device(device):
    model_simple = keras.Sequential()

    # model_lstm.add(layers.LSTM(units=128, input_shape=[1, 3]))
    model_simple.add(layers.Input(shape=(100, 3, 1), name="sensor_image"))
    model_simple.add(layers.ZeroPadding2D(padding=(0, 2)))
    model_simple.add(layers.Conv2D(8, kernel_size=(3, 3), strides=(1, 1), activation="relu"))
    model_simple.add(layers.Conv2D(16, kernel_size=(2, 2), strides=(1, 1), activation="relu"))
    model_simple.add(layers.Conv2D(16, kernel_size=(2, 2), activation="relu"))
    model_simple.add(layers.MaxPooling2D(pool_size=(3, 1)))
    model_simple.add(layers.Conv2D(16, kernel_size=(5, 1), activation="relu"))
    model_simple.add(layers.Flatten())
    model_simple.add(layers.Dropout(0.5))
    model_simple.add(layers.Dense(len(Gestures), activation="softmax", name="predictions"))
    model_simple.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc', 'mae'])
    model_simple.summary()


    start = time.perf_counter()
    simple_history = model_simple.fit(x_train_normalized, y_train, epochs=80, batch_size=2)

    end = time.perf_counter()
    print("Training took: %0.1f seconds" % (end - start))

# Training times:
| GPU   | CPU   |
|-------|-------|
| 18.5s | 12.3s |

In [ ]:
loss = simple_history.history['loss']
mae = simple_history.history['mae']

epochs = range(1, len(loss) + 1)

fig, axs = plt.subplots(1, 2, figsize=(15, 5))

axs[0].plot(epochs, loss, 'g.', label='Training Loss')
axs[0].set_xlabel('Epochs')
axs[0].set_ylabel('Loss')
axs[0].legend()

axs[1].plot(epochs, mae, 'g.', label='Training Accuracy')
axs[1].set_xlabel('Epochs')
axs[1].set_ylabel('MAE')
axs[1].legend()

fig.show()

In [ ]:
score = model_simple.evaluate(x_test_normalized, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

In [ ]:
print(x_test_normalized.shape)

for i in range(x_test_normalized.shape[0]):
    test_sample = np.expand_dims(x_test_normalized[i], -4)
    test_prediction = model_simple.predict(test_sample)
    print("Prediction: %s, actual: %s" % (Gestures(np.argmax(test_prediction)), Gestures(np.argmax(y_test[i]))))

In [ ]:
# Split data


In [ ]:
USE_GPU = True
device = "/device:GPU:0" if USE_GPU else "/device:CPU:0"

with tf.device(device):
    # https://towardsdatascience.com/time-series-classification-for-human-activity-recognition-with-lstms-using-tensorflow-2-and-keras-b816431afdff
    model_lstm_stateless = keras.Sequential(name="LSTM_Stateless")

    # Subject to change as we split up the data in fragments:
    # Stateless?
    model_lstm_stateless.add(layers.Input(shape=(100, 3), name="sensor_image"))
    model_lstm_stateless.add(layers.LSTM(units=128, name="lstm"))
    # Stateful?
    # model_lstm_stateless.add(layers.Input(batch_shape=(100, 3, 1), name="sensor_image"))
    # model_lstm_stateless.add(layers.LSTM(units=128, stateful=True, name="lstm"))

    # Extra
    model_lstm_stateless.add(layers.Dropout(rate=0.5, name="dropout"))
    model_lstm_stateless.add(layers.Dense(units=128, activation='relu', name="dense_1"))

    # Output stage
    model_lstm_stateless.add(layers.Dense(len(Gestures), activation="softmax", name="predictions"))
    model_lstm_stateless.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc', 'mae'])
    model_lstm_stateless.summary()

    start = time.perf_counter()

    # Do we need to reset the states inbetween?
    lstm_stateless_history = model_lstm_stateless.fit(x_train_normalized, y_train, epochs=1000, batch_size=1, shuffle=False)

    end = time.perf_counter()
    print("Training took: %0.1f seconds" % (end - start))

In [ ]:
plot_history(lstm_stateless_history)

In [ ]:
score = model_lstm_stateless.evaluate(x_test_normalized, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

In [ ]:
print("================= Running on training data (Toy Example): =================")
for i in range(x_train_normalized.shape[0]):
    test_sample = np.expand_dims(x_train_normalized[i], -4)
    test_prediction = model_lstm_stateless.predict(test_sample)
    print("Prediction: %s, actual: %s" % (Gestures(np.argmax(test_prediction)), Gestures(np.argmax(y_train[i]))))

print("\n\n================= Running on testing data: =================")
for i in range(x_test_normalized.shape[0]):
    test_sample = np.expand_dims(x_test_normalized[i], -4)
    test_prediction = model_lstm_stateless.predict(test_sample)
    print("Prediction: %s, actual: %s" % (Gestures(np.argmax(test_prediction)), Gestures(np.argmax(y_test[i]))))

In [ ]:
USE_GPU = True
device = "/device:GPU:0" if USE_GPU else "/device:CPU:0"

with tf.device(device):
    # https://towardsdatascience.com/time-series-classification-for-human-activity-recognition-with-lstms-using-tensorflow-2-and-keras-b816431afdff
    model_lstm_stateful = keras.Sequential(name="LSTM_Stateful")

    # Subject to change as we split up the data in fragments:
    model_lstm_stateful.add(layers.Input(batch_shape=(100, 3, 1), name="sensor_image"))
    model_lstm_stateful.add(layers.LSTM(units=128, stateful=True, name="lstm"))

    # Extra
    model_lstm_stateful.add(layers.Dropout(rate=0.5, name="dropout"))
    model_lstm_stateful.add(layers.Dense(units=128, activation='relu', name="dense_1"))

    # Output stage
    model_lstm_stateful.add(layers.Dense(len(Gestures), activation="softmax", name="predictions"))
    model_lstm_stateful.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc', 'mae'])
    model_lstm_stateful.summary()

    start = time.perf_counter()

    # Do we need to reset the states inbetween?
    lstm_stateful_history = model_lstm_stateful.fit(x_train_normalized, y_train, epochs=1000, batch_size=1, shuffle=False)

    end = time.perf_counter()
    print("Training took: %0.1f seconds" % (end - start))

In [ ]:
plot_history(lstm_stateful_history)